In [4]:
!pip install --upgrade transformers -q
!pip install accelerate -q

# GPTQ Dependencies
!pip install --upgrade optimum -q
!pip install --upgrade auto-gptq -q

# RAG Dependencies
!pip install langchain -q
!pip install pypdf -q
!pip install html2text -q
# !pip install torch==1.2.0 torchvision==0.4.0 -f -q
!pip install -U sentence-transformers -q
!pip install faiss-cpu -q

# BERT
!pip install bert-extractive-summarizer -q
# !pip install spacy==2.0.12 -q

In [15]:
from langchain.vectorstores import FAISS
from langchain.document_loaders.pdf import PyPDFDirectoryLoader
from langchain.document_transformers import Html2TextTransformer
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from summarizer import Summarizer
from pathlib import Path
import nest_asyncio

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import sys, json, re

PATH="/content/game.txt"
model_id = "TheBloke/Mistral-7B-Instruct-v0.2-GPTQ"
revision = "gptq-4bit-32g-actorder_True"

In [19]:
tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    padding=True,
    padding_side = "left",
    use_fast=True
)
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [8]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    trust_remote_code=False,
    revision=revision
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.57G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4371: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(


generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [20]:
# Create a pipeline
pipe = pipeline(
    task='text-generation',
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True, # creative generation by discouraging greedy decoding
    temperature=1,
    top_p=0.95,
    top_k=40,
    repetition_penalty=1.1,
    return_full_text = False  # Only return the current output instead of returning complete prompt
)

In [10]:
bert_model = Summarizer()

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [29]:
def make_vdb(path):
  loader = TextLoader(path)
  doc=loader.load()

  # Chunk text
  text_splitter = CharacterTextSplitter(chunk_size=10, chunk_overlap=0)
  chunked_documents = text_splitter.split_documents(doc)

  embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2')

  # Load chunked documents into the FAISS index
  db = FAISS.from_documents(chunked_documents, embeddings)

  # Connect query to FAISS index using a retriever
  retriever = db.as_retriever(
      search_type="similarity",
      search_kwargs={'k': 3}
  )

  folder_path = Path("/content/faiss_index")

  if folder_path.exists():
    old_db = FAISS.load_local("/content/faiss_index", embeddings,allow_dangerous_deserialization=True)
    db.merge_from(old_db)
    db.save_local("/content/faiss_index")
    return db
  else:
    db.save_local("/content/faiss_index")
    return db

def make_rag_query(query, db):
  docs = db.similarity_search(query)
  result=docs[0].page_content
  return result

def save_to_txt(content):
    filename="/content/game.txt"
    with open(filename, 'w') as file:
        file.write(content)

def rag_optimize(context,query):
    print("context: ", context, type(context))
    system = f"""
    You are excellent at creating simplified statemnet about the context given to you. Use this simplified statement to answer the question [{query}]. The answer must be concise.
    [{context}]

    """

    # one-shot prompting
    chat = [
      {"role": "user", "content": system}
    ]

    # prepare the prompt using the chat template
    prompt = tokenizer.apply_chat_template(chat, tokenize=False)
    # run the pipeline to generate the model output
    outputs = pipe(prompt)
    output = outputs[0]["generated_text"].strip()
    return output

def summarize_scenerio(scenerio):
  bert_summary = ''.join(bert_model(scenerio, min_length=10))
  return bert_summary

def extract_first_json(text):
    # Define a regex pattern to match the first JSON object
    pattern = r'{\s*".*?"\s*:\s*{.*?}\s*}'

    # Use re.search to find the first match of the pattern in the text
    match = re.search(pattern, text, re.DOTALL)

    if match:
        # Extract and return the matched JSON object
        return match.group()
    else:
        return None

# This function takes in the malformed json and creates a prompt for the model to convert it into a valid json as per the given valid json schema.
def json_fixer(malformed_json, valid_json_schema):
    prompt = f"""Generate valid JSON from the malformed JSON fixing missing commas, quotes and brackets.

valid JSON should strictly follow the following "json-valid" schema:
```json-valid
{valid_json_schema}
```

Here is a malformed json:
```json-malformed
{malformed_json}
```

Here is a fixed JSON, with fixed missing commas, quotes and brackets:
```json"""

    return prompt

In [41]:
health = 20
story = ""

# used for one-shot prompting to let the model know the output structure
option_assistant = f"""
  {{
    "option1": {{
      "text": "This is choice 1",
      "outcome": "This is the narrative for choice 1",
      "damage": 0
    }},
    "option2": {{
      "text": "This is choice 2",
      "outcome": "This is the narrative for choice 2",
      "damage": 5
    }},
    "option3": {{
      "text": "This is choice 3",
      "outcome": "This is the narrative for choice 3",
      "damage": 0
    }}
  }}
  """

while health > 0:
  # prompt for story generation
  story_prompt = f"""
  You are an AI dungeon master that provides any kind of roleplaying game content.

  Instructions:

  - Be specific, descriptive, and creative.
  - Avoid repetition and avoid summarization.
  - Generally use second person (like this: 'He looks at you.'). But use third person if that's what the story seems to follow.
  - Never decide or write for the user. If the input ends mid sentence, continue where it left off.
  - Make sure you always give responses continuing mid sentence even if it stops partway through.

  {story}

  """

  # run the pipeline to generate the narrative based on the story so far
  outputs = pipe(story_prompt)
  narrative = outputs[0]["generated_text"].strip()
  print(health)
  print(narrative)

  # prompt for generating branching narratives and the choices along with the associated damage score based on the most recent narrative
  option_user = f"""
  You are an expert interactive fiction writer who specializes in crafting short and creative branching narratvies.\
  Create three branching narratives for the story excerpt provided below.\
  The generated narrative should be one-liner sentences with less than 20 words referencing the keywords from the story and highlighting the key details.\
  Also, present each narrative in the form of user-visible choice as well. The choices must be of a few words capturing the essence of the resulting narrative.\
  Additionally, associate damage score with each choice: 5 damage if selecting this choice can bring damage, otherwise 0 damage.
  Generate only one JSON object containing the narratives and corresponding choices using the following json schema.

  ```json
  {{
    "option1": {{
      "text": "This is choice 1",
      "outcome": "This is the narrative for choice 1",
      "damage": 0
    }},
    "option2": {{
      "text": "This is choice 2",
      "outcome": "This is the narrative for choice 2",
      "damage": 5
    }},
    "option3": {{
      "text": "This is choice 3",
      "outcome": "This is the narrative for choice 3",
      "damage": 0
    }}
  }}

  Story Excerpt:

  {narrative}

  """

  chat = [
      {"role": "user", "content": option_user},
      {"role": "assistant", "content": option_assistant},
      {"role": "user", "content": "```json"},
  ]

  # run the pipeline to generate the choices and associated damage score
  option_prompt = tokenizer.apply_chat_template(chat, tokenize=False)
  outputs = pipe(option_prompt)
  output = outputs[0]["generated_text"].strip()

  try:
    data = json.loads(extract_first_json(output))
  except:
    data = None

  if data is None:
    print("fixing JSON!!!")
    prompt = json_fixer(output, option_assistant)
    outputs = pipe(prompt)
    output = outputs[0]["generated_text"].strip()
    data = json.loads(extract_first_json(output))

  print("option1", data['option1']['text'])
  print("option1", data['option1']['damage'])

  print("option2", data['option2']['text'])
  print("option2", data['option2']['damage'])

  print("option3", data['option3']['text'])
  print("option3", data['option3']['damage'])

  user_input = input("> ")
  print("")

  # RAG
  save_to_txt(narrative)
  db=make_vdb(PATH)

  if user_input not in ['option1', 'option2', 'option3']:
    result = make_rag_query(user_input, db)
    optimize_res = rag_optimize(result, user_input)
    print("RAG response: ",optimize_res)
  else:
    outcome = data[user_input]['outcome']
    damage = data[user_input]['damage']
    story += narrative + outcome
    health -= damage
    print(outcome)

  # summary = summarize_scenerio(narrative)

20
The sun sets behind you, painting the sky with hues of orange, pink, and red. You find yourself on a cliff overlooking the endless expanse of water - the Sapphire Sea. Waves crash against the jagged cliffs beneath you, sending plumes of spray into the air. Seabirds call in the distance as they glide over the ocean. In your hand, you clutch the handle of an ancient key, passed down from generation to generation in your family. It's said that this key unlocks the legendary treasure hidden deep within the Blue Shrine. Your journey has finally brought you here.

  As night descends upon the sea, you can hear the lullaby-like sound of crickets in the forest near the shrine. Their melodies merge with the rhythmic sounds of the waves, creating a soothing symphony that calms your nerves.

  A faint breeze caresses your face, carrying the scent of saltwater and wildflowers. In the distance, you see a sliver of moonlight reflecting off the surface of the water, drawing closer to the shore.

 

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


You charge at the woman, brandishing the ancient key menacingly. She reacts defensively, triggering a magical battle.
15
OR:
  
  You approach the woman cautiously, unsure of her intentions.

  She smiles warmly, revealing no signs of malice or fear. "Fear not, brave adventurer," she says, her voice soft and calming. "I am but a guardian of these sacred lands, protecting them from those who would misuse their power."

  She takes the ancient key from your hand, examining it intently. "This is indeed the key to the Blue Shrine," she admits, her eyes narrowing slightly. "But only those pure of heart may enter and claim the treasure contained therein."

  She looks up at you, appraising your soul with her magically gifted gaze. "Your heart is strong, but there is a darkness within you that must be purged before you may pass."

  She offers you a choice:

  1) Spend time within her sanctuary, learning the ways of her magical art and purging your darkness;
  2) Return to your village and re

KeyboardInterrupt: 

In [40]:
(data)

{'option1.1': {'text': 'Step through arch.',
  'outcome': "The griffon's eyes glow with ethereal light, guiding you deeper into the heart of Khalidor. Uncovering relics, you gain insight into the ancient civilization.",
  'damage': 0},
 'option1.2': {'text': 'Shun the call of the arch.',
  'outcome': 'You retreat, leaving behind the mysteries of Khalidor, living to tell the tale and learn from the experience.',
  'damage': 0}}